In [2]:
import keras
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import np_utils
from bayes_opt import BayesianOptimization

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
df_train = pd.read_csv('https://github.com/ozt-ca/tjo.hatenablog.samples/raw/master/r_samples/public_lib/jp/exp_uci_datasets/online_news_popularity/ONP_train.csv')
df_test = pd.read_csv('https://github.com/ozt-ca/tjo.hatenablog.samples/raw/master/r_samples/public_lib/jp/exp_uci_datasets/online_news_popularity/ONP_test.csv')

x_train = df_train.iloc[:, :58]
y_train = df_train["shares"]
x_test = df_test.iloc[:, :58]
y_test = df_test["shares"]

merged_train = pd.concat([x_train, x_test])

scaler = StandardScaler()
merged_train.iloc[:, 0:10] = scaler.fit_transform(merged_train.iloc[:, 0:10])
merged_train.iloc[:, 17:28] = scaler.fit_transform(merged_train.iloc[:, 17:28])
merged_train.iloc[:, 37:] = scaler.fit_transform(merged_train.iloc[:, 37:])

x_train = merged_train.iloc[:len(df_train), :]
x_test = merged_train.iloc[len(df_train):, :]

In [10]:
df_train = shuffle(df_train)
idx = int(np.round(len(df_train) * 0.9, 0))
x_ptrain = df_train.iloc[:idx, :58]
y_ptrain = df_train.iloc[:idx, 58]
x_ptest = df_train.iloc[idx:, :58]
y_ptest = df_train.iloc[idx:, 58]

In [30]:
def get_model(unit1, unit2, unit3):
    model = Sequential()
    model.add(Dense(int(unit1), input_dim = 58))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(int(unit2)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(int(unit3)))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('linear'))
    return model

def fit_with(unit1, unit2, unit3, lr, num_epoch):
    model = get_model(unit1, unit2, unit3)
    sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss = 'mean_squared_error', optimizer = sgd)
    model.fit(x_ptrain, y_ptrain, epochs = int(num_epoch), batch_size = 200, verbose = 0)
    pred = model.predict(x_ptest)
    rmse = np.sqrt(mean_squared_error(pred, y_ptest))
    return -rmse

pbounds = {'unit1': (200, 300), 'unit2': (120, 160), 'unit3': (60, 80),
           'lr': (1e-3, 1e-2), 'num_epoch': (50, 100)}

optimizer = BayesianOptimization(
    f = fit_with,
    pbounds = pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=5, n_iter=5,)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.res['max']['max_params'])


Initialization
-------------------------------------------------------------------------------------------
 Step |   Time |      Value |        lr |   num_epoch |     unit1 |     unit2 |     unit3 | 
    1 | 01m02s |   -0.91264 |    0.0082 |     83.5234 |  241.9195 |  123.6935 |   68.3404 | 
    2 | 00m56s |   -0.91318 |    0.0097 |     70.8652 |  268.5220 |  127.4504 |   74.4065 | 
    3 | 00m59s |   -0.91554 |    0.0038 |     77.9345 |  220.4452 |  133.8224 |   60.0023 | 
    4 | 00m47s |   -0.91168 |    0.0072 |     57.0193 |  287.8117 |  135.8707 |   66.0467 | 
    5 | 00m44s |   -0.91220 |    0.0089 |     59.9051 |  202.7388 |  141.5527 |   62.9351 | 
Bayesian Optimization
-------------------------------------------------------------------------------------------
 Step |   Time |      Value |        lr |   num_epoch |     unit1 |     unit2 |     unit3 | 
    6 | 00m47s |   -0.91586 |    0.0074 |     50.5080 |  220.3130 |  120.0727 |   60.5185 | 
    7 | 01m48s |   -0.91177 |    0.

In [31]:
model = Sequential()
model.add(Dense(int(optimizer.res['max']['max_params']['unit1']), input_dim = 58))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(int(optimizer.res['max']['max_params']['unit2'])))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(int(optimizer.res['max']['max_params']['unit3'])))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('linear'))

sgd = SGD(lr=optimizer.res['max']['max_params']['lr'], decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'mean_squared_error', optimizer = sgd)
model.fit(x_train, y_train, epochs = int(optimizer.res['max']['max_params']['num_epoch']), batch_size = 200, verbose = 0)
pred = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(pred, y_test))
print "RMSE:", rmse

RMSE: 0.9372867159486481
